# Datat overview

В данном ноутбуке производится обзор имеющихся данных (документов, запросов и sample submission), а также перезапись документов в формат .pkl по номерами их docid.

In [1]:
import os
from tqdm.notebook import tqdm
import pickle
from bs4 import BeautifulSoup
import re
from pymystem3 import Mystem
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

PATH = "/Users/michelle/data/text-relevance-competition-ir-1-ts-fall-2020/"

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/michelle/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [82]:
mystem = Mystem()
stop_list = stopwords.words("russian")

In [85]:
with open(os.path.join(PATH, "stop_words.pkl"), "wb") as f:
    pickle.dump(stop_list, f)

In [3]:
with open(PATH + "urls.numerate.txt", "r") as f:
    urls = f.read().split("\n")

In [4]:
len(urls)

38115

In [7]:
urls[:100]

['1\teva.ru/forum/topic-tree-mobile.htm?messageId=89211158&topicId=3285169',
 '2\te1.ru/articles/travel/page_4/009/994/article_9994.html',
 '3\tbikepost.ru/qa/post/9065/Kak-pravilno-sdelat-vizu-v-Polshu-shengen.html',
 '4\trutax.polpred.com/?ns=1&page=2',
 '5\tcoollib.com/b/259649/read',
 '6\tcoollib.com/b/244202/read',
 '7\trabotatut.do.am/index/o_vizakh/0-5',
 '8\tkaliningrad.zoon.ru/hotels/turisticheskoe_agentstvo_dyuna39/social',
 '9\tforum.materinstvo.ru/lofiversion/index.php/t1358686',
 '10\tbookslist.me/read2.php?id=8834',
 '11\tlifecz.ru/threads/1375/page-111',
 '12\tstudenttests.ru/%D1%83%D0%BD%D0%B8%D0%B2%D0%B5%D1%80%D1%81%D0%B8%D1%82%D0%B5%D1%82%D1%8B/Niderlandy',
 '13\tgulfcoolcontracting.com/uploads/userfiles/file/dom8197-sertifikat-na-pokupku-bileta-v-teatr-jam7575.xml',
 '14\tdigesttour.ru/node/749',
 '15\tforum.immigration.cz/archive/index.php/t-5389.html',
 '16\tspaininvestor.com/Views/TextosWebForm.aspx?IdTexto=379',
 '17\tsarov24.ru/mneniya/640-bezvizovye-strany-2017

In [4]:
with open(PATH + "queries.numerate.txt", "r") as f:
    queries = f.read().split("\n")

In [10]:
len(queries)

400

In [10]:
queries[:100]

['1\tмультивиза в израиль какие страны можно посетить',
 '2\tнадо ли носить компрес гольфы после операции на голеностопе',
 '3\tжировики на спине можно ли применить пиявки',
 '4\trfr ghjgbcfnm ghjcnj flvbre',
 '5\tхто буде судити суперкубок 2017 між шахтар динамо який відбудеться в одесі',
 '6\tкак доехать от метро мякинино до крокус сити молл',
 '7\tкак заменить подшипник ступицы ваз 2121',
 '8\tкак установить мод сумеречный лес на майнкрафт 1 12',
 '9\tможно ли ставить горчичник при ларингите',
 '10\tсамый точный прогноз погоды в балашове',
 '11\tчто нейтрализует ожог борщевика',
 '12\tсмородина черная протертая сколько нужно сахара',
 '13\tкак подключить роутер',
 '14\tот воды зуд в руках почему',
 '15\tсамый компактный пистолет в ссср',
 '16\tкак подключить двухконтурный газовый котел видео',
 '17\tесли есть один раз вечером можно похудеть',
 '18\tкаквернуть налог в аэропорту парижа',
 '19\tкак добраться до деревни бобруйск васьково',
 '20\tна какой день цикла сдают т3',
 '21\tчто 

In [5]:
with open(PATH + "sample.technosphere.ir1.textrelevance.submission.txt", "r") as f:
    sample = f.read().split("\n")

Считаем количество пар запрос-документ и общее количество документов в базе

In [20]:
query_set = set()
for item in sample:
    if "," in item:
        query_set.add(item.split(",")[1])
print(len(query_set) - 1) # headers

38114


In [15]:
counter = 0
for folder in os.listdir(PATH + "content/content/"):
    if folder.startswith("20"):
        counter += len(os.listdir(PATH + "content/content/" + folder))
print(counter)

38114


По урлам сопоставим каждому документу его docid и сохраним в соответствующий файл.

In [6]:
urls_docids = {item.split("\t")[1]: int(item.split("\t")[0]) for item in urls if len(item)}

In [8]:
error_counter = 0
error_list = []
for folder in os.listdir(os.path.join(PATH, "content/content")):
    if folder.startswith("20"):
        filelist = os.listdir(os.path.join(PATH, "content/content", folder))
        for filename in tqdm(filelist, total=len(filelist)):
            with open(os.path.join(PATH, "content/content", folder, filename), "r") as f:
                try:
                    doc = f.read()
                except Exception as e:
                    error_counter += 1
                    error_list.append(os.path.join(folder, filename))
                    continue
            url = doc.split("\n")[0]
            docid = urls_docids[url]
            with open(os.path.join(PATH, "content", f"{docid}.pkl"), "wb") as f:
                pickle.dump(doc, f)
            os.remove(os.path.join(PATH, "content/content", folder, filename))
            
print(error_counter)


595


In [9]:
for filename in error_list:
    with open(os.path.join(PATH, "content/content", filename), "rb") as f:
        doc = f.read().decode(errors='ignore')
    url = doc.split("\n")[0]
    docid = urls_docids[url]
    with open(os.path.join(PATH, "content", f"{docid}.pkl"), "wb") as f:
        pickle.dump(doc, f)

In [11]:
pkl_counter = 0
for filename in os.listdir(os.path.join(PATH, "content")):
    pkl_counter += int(filename.endswith(".pkl"))
print(pkl_counter)

38114
